In [30]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import csv
import sys 

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from src import helpers, preprocessing, exploration, polynomial_exp
from src.helpers import split_data_rand
from src.helpers import build_model_data
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from src.preprocessing import undefined_to_median, undefined_to_mean, prune_undefined
from imblearn.combine import SMOTEENN
pd.set_option("display.max_column", 100)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
# x_df = pd.read_csv("../data/raw/x_train.csv")
# y_df = pd.read_csv("../data/raw/y_train.csv")
df = pd.read_csv("../data/external/heart_disease_health_indicators_BRFSS2015.csv")
df

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,0.0,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0
253676,0.0,1.0,1.0,1.0,18.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0
253677,0.0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0
253678,0.0,1.0,0.0,1.0,23.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,5.0,1.0


In [57]:
# preprocessing
seed = 212
degree = 1

# x = x_df.values
# y = y_df["_MICHD"].values

df = df.drop('Stroke', axis=1)
df = df.drop('BMI', axis=1)
df = df.drop('Education', axis=1)
df = df.drop('CholCheck', axis=1)
df = df.drop('AnyHealthcare', axis=1)
df = df.drop('HvyAlcoholConsump', axis=1)
x = df.drop('HeartDiseaseorAttack', axis=1).values
y = df.HeartDiseaseorAttack.values.astype(int)

positive_indices = np.where(y == 1)[0]
negative_indices = np.where(y != 1)[0]

min_samples = min(len(positive_indices), len(negative_indices))

random.seed(seed)
downsampled_negative_indices = random.sample(list(negative_indices), min_samples)

balanced_indices = np.concatenate([positive_indices, downsampled_negative_indices])
random.shuffle(balanced_indices)

balanced_x = x[balanced_indices]
balanced_y = y[balanced_indices]

split_ratio = 0.8  # 80% train, 20% test
split_index = int(len(balanced_x) * split_ratio)

x_train = balanced_x[:split_index]
y_train = balanced_y[:split_index]
x_test = balanced_x[split_index:]
y_test = balanced_y[split_index:]

# x_train = preprocessing.clean_features(x_train)
# x_test = preprocessing.clean_features(x_test)

# x_train = undefined_to_mean(x_train)
# x_train = helpers.standardize(x_train)
# print(x_train.shape)

# x_test = undefined_to_mean(x_test)
# x_test = helpers.standardize(x_test)
# print(x_test.shape)

In [58]:
model = LogisticRegression(penalty=None)
model.fit(x_train, y_train)
print("Accuracy: " + str(model.score(x_test, y_test)))
pred = model.predict(x_test)
print("F1: " + str(f1_score(y_test, pred)))

Accuracy: 0.7595731324544884
F1: 0.7685334407735698


/Users/francoisdumoncel/anaconda3/envs/ada/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
x_te = pd.read_csv("../data/raw/x_test.csv")
ids = x_te.Id
x_te = preprocessing.clean_features(x_te.values)
x_te = undefined_to_median(x_te)
x_te = helpers.standardize(x_te)
preds = model.predict(x_te)
#create csv 
helpers.create_csv_submission(ids, preds, "test.csv")

[-1 -1 -1 ... -1 -1 -1]


In [ ]:
1. Lire les données (helpers)

2. Clean les données (preprocessing)

3. Train un model (fitters)

3bis. Afficher des métriques (metrics)

4. Prédire et fournir le CSV (helpers)